##### Copyright 2018 The TensorFlow Authors.



In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 시각적 초점을 통한 이미지 캡션

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/image_captioning">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/tutorials/text/image_captioning.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    구글 코랩에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/text/image_captioning.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    GitHub 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ko/tutorials/text/image_captioning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 다운로드하기</a>
  </td>
</table>


우리의 목표는 아래의 예시 이미지가 주어졌을 때, "파도를 타는 서퍼"와 같은 캡션을 만드는 것 입니다.

![Man Surfing](https://tensorflow.org/images/surf.jpg)

*[이미지 소스](https://commons.wikimedia.org/wiki/Surfing#/media/File:Surfing_in_Hawaii.jpg); 라이선스 : 공개 라이선스*

작업을 완수하기 위해 모델이 캡션을 생성할 때 어떤 부분에 초점을 맞추는지 확인할 수 있는 초점 기반 모델을 사용할 것 입니다.

![Prediction](https://tensorflow.org/images/imcap_prediction.png)

이 모델 구조는 [Show, Attend and Tell: Neural Image Caption Generation with Visual Attention](https://arxiv.org/abs/1502.03044)와 유사합니다.

 이 노트북은 End-to-End 방식 학습 예제입니다. 노트북을 실행하면, [MS-COCO](http://cocodataset.org/#home) 데이터 세트를 다운로드 하고, Inception V3를 사용하여 이미지의 일부를 전처리 및 캐시하고, 인코더-디코더 모델을 훈련시키고, 그 훈련된 모델을 사용하여 새로운 이미지에 대한 캡션을 생성합니다.

아래의 예제에서는 약 20,000개의 이미지에 대한 30,000개의 캡션 정도의 비교적 적은 양의 데이터에 대한 모델을 훈련할 것 입니다(하나의 이미지당 여러 개의 캡션이 있기 때문에).

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [9]:
try:
  # %tensorflow_version은 오직 코랩에만 존재합니다.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
# 캡션하는 동안 모델이 이미지의 어느 부분에 초점을 맞추는지 확인하기 위해
# 초점플롯을 생성합니다.
import matplotlib.pyplot as plt

# Scikit-learn은 많은 유용한 유틸리티를 포함합니다.
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

ModuleNotFoundError: No module named 'tensorflow'

## MS-COCO 데이터세트 다운로드 및 준비

모델을 훈련시키기 위해 [MS-COCO 데이터세트](http://cocodataset.org/#home)를 사용합니다. 이 데이터 세트는 각각 최소 5개의 캡션을 가진 82,000개 이상의 이미지를 가지고 있습니다. 아래 코드는 데이터세트를 자동으로 다운로드 및 추출합니다.

**주의: 대용량 다운로드**. 훈련 세트 파일의 용량은 13GB입니다.

In [5]:
annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'

name_of_zip = 'train2014.zip'
if not os.path.exists(os.path.abspath('.') + '/' + name_of_zip):
  image_zip = tf.keras.utils.get_file(name_of_zip,
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip)+'/train2014/'
else:
  PATH = os.path.abspath('.')+'/train2014/'

NameError: name 'tf' is not defined

## 훈련 세트의 크기 제한
본 튜토리얼에 대한 훈련을 가속하기 위해, 30,000개의 캡션과 그에 상응하는 이미지를 사용해 모델을 훈련시킬 것 입니다. 더 많은 데이터를 선택할수록 캡션의 품질이 향상될 것 입니다.

In [0]:
# json 파일을 읽습니다.
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# 벡터에 캡션과 이미지 이름을 저장
all_captions = []
all_img_name_vector = []

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

# 이미지 이름과 캡션을 함께 셔플합니다
# 임의의 상태로 설정합니다
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# 셔플된 세트의 상위 30,000 캡션만 선택합니다
num_examples = 30000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

In [0]:
len(train_captions), len(all_captions)

## InceptionV3를 사용한 이미지 전처리

다음으로, 이미넷에 사전 훈련된 InceptionV3를 사용해 각 이미지를 분류합니다. 마지막 컨볼루셔널 레이어에서 특성을 추출합니다.

첫번째로, 이미지를 InceptionV3가 요구하는 형식으로 이미지를 변환 합니다 :
* 이미지 사이즈를 299px * 299px로 재설정
*  InceptionV3를 훈련하는데 사용되는 이미지의 형식과 일치하는 -1 ~ 1 범위의 픽셀을 포함하도록 정규화하는 [전처리 입력](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input) 기능을 통해 [이미지 전처리](https://cloud.google.com/tpu/docs/inception-v3-advanced#preprocessing_stage)를 진행하세요.


In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

## InceptionV3 초기화 및 사전 훈련된 ImageNet 가중치 로드
이제 InceptionV3 아키텍처의 출력 레이어의 마지막 컨볼루셔널 레이어인 tf.keras 모델을 생성할 것 입니다. 이 레이어의 출력 형태는 ```8x8x2048``` 입니다. 이 예제에서는 '초점' 을 사용 하기 때문에 마지막 컨볼루셔널 레이어를 사용합니다. 이러한 초기화는 병목 현상을 유발할 수 있으므로 훈련중에는 초기화하지 않습니다.

* 각 이미지를 네트워크를 통해 전달하고 결과 벡터를 딕셔너리에 저장합니다(image_name --> feature_vector).
* 모든 이미지가 네트워크를 통해 전달되면, 딕셔너리를 선택하고 디스크에 저장합니다.


In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

## InceptionV3 에서 추출한 특성 캐싱
InceptionV3로 각 이미지를 전처리하고, 출력을 디스크에 캐싱합니다. RAM에서 출력을 캐싱하는 것은 더 빠를 뿐만 아니라, 메모리 집약적이어서 이미지당 8 \* 8 \* 2048 의 'float'을 요구합니다. 이것은 Colab의 메모리 한계를 초과합니다(현재 12GB).
보다 정교한 캐싱 전략(랜덤 액세스 디스크 I/O를 줄이기 위해 이미지를 샤딩 하는 등)을 통해 성능을 개선할 수 있지만, 그렇게 하기 위해서는 더 많은 코드를 필요로 할 것 입니다.

캐싱은 GPU와 함께 Colab에서 약 10분가량 실행 되는데, 이때 진행 표시줄을 보고싶다면 다음을 참고하세요 :

1. [tqdm](https://github.com/tqdm/tqdm) 설치 :

    `!pip install tqdm`

2. tqdm 패키지 가져오기 :

    `from tqdm import tqdm`

3. 다음의 라인 변경 :

    `for img, path in image_dataset:`

    에서 아래와 같이:

    `for img, path in tqdm(image_dataset):`


In [1]:
# 고유한 이미지 얻기
encode_train = sorted(set(img_name_vector))

# 시스템 구성에 따라 batch_size를 자유롭게 조절할 수 있습니다
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

NameError: name 'img_name_vector' is not defined

## 캡션 전처리 및 토큰화

* 먼저 캡션을 토큰화 하세요(예 : 공간 분할). 이는 데이터의 모든 고유한 단어들의 어휘를 제공합니다. (예 : "서핑", "풋볼" 등)
* 다음으로, 어휘의 사이즈를 상위 5,000개로 제한합니다. (메모리 절약을 위해).
* 다른 모든 단어들을 "알 수 없는(Unknown)" 토큰으로 대체합니다..
* 이후 단어-인덱스 및 인덱스-단어 매핑을 생성합니다.
* 마지막으로, 모든 시퀀스를 가장 긴 시퀀스와 동일한 길이로 패딩합니다.

In [2]:
# 데이터 세트 내의 최대 길이 캡션 찾기
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# 단어들 중 상위 5,000개 단어 선택
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [0]:
# 토큰화된 벡터 생성
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
# 각 벡터를 캡션의 max_length 로 패딩
# max_length값을 제공하지 않을 경우, pad_sequences에서 자동으로 계산합니다.
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [3]:
# 초점 가중치를 저장하는데 사용하는 max_length 계산
max_length = calc_max_length(train_seqs)

NameError: name 'train_seqs' is not defined

## 교육 및 테스트를 통한 데이터 분할

In [5]:
# 80-20 분할을 사용하여 훈련 및 검증 세트 생성
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)

NameError: name 'train_test_split' is not defined

In [0]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

## 훈련을 위한 tf.data 데이터 세트 생성

이미지와 캡션이 준비되었습니다! 다음으로, 모델 훈련에 사용할 tf.data 데이터 세트를 생성해봅시다.

In [0]:
# 시스템 구성에 따라 매개변수를 자유롭게 변경할 수 있습니다.

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index) + 1
num_steps = len(img_name_train) // BATCH_SIZE
# InceptionV3에서 추출한 벡터의 형태는 (64, 2048)
# 두 변수는 벡터 형태를 나타냅니다.
features_shape = 2048
attention_features_shape = 64

In [0]:
# Numpy 파일 로드
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# 맵을 사용하여 Numpy 파일을 병렬로 로드
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# 셔플 및 묶기
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

NameError: name 'tf' is not defined

## 모델
재미있는 사실 : 아래의 디코더는 [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb) 의 예제와 동일합니다.

이 모델 아키텍처는 [Show, Attend and Tell](https://arxiv.org/pdf/1502.03044.pdf) 에 영감을 받아 제작되었습니다.

* 이 예제에서, InceptionV3의 낮은 단계 컨볼루셔널 레이어에서 특성을 추출하여 (8, 8, 2048) 형태의 벡터를 제공합니다. 
* 그 벡터를 (64, 2048) 과 같은 형태로 밀어 넣습니다.
* 이 벡터는 CNN 인코더(완전 연결 단일 레이어로 구성된)를 통해 전달됩니다.
* RNN (여기서는 GRU) 는 다음 단어를 예측하기 위해 이미지를 검토합니다.

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # 특성(CNN_encoder 출력) 형태 == (batch_size, 64, embedding_dim)

    # 은닉 형태 == (batch_size, hidden_size)
    # hidden_with_time_axis 형태 == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # 점수 형태 == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights 형태 == (batch_size, 64, 1)
    # self.V에 점수를 적용하고 있기 때문에 마지막 축에서 1을 얻습니다.
    
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # 덧셈 후의 context_vector 형태 == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [7]:
class CNN_Encoder(tf.keras.Model):
    # 이미 형상을 추출하고 pickle을 이용해 덤프하였으므로
    # 이 인코더는 완전 연결 레이어를 통해 특성을 전달합니다.
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # fc 후의 형태 == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

NameError: name 'tf' is not defined

In [0]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # 초점을 별도의 모델로 정의
    context_vector, attention_weights = self.attention(features, hidden)

    # 임베딩 후의 x의 형태 == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # 결합 후의 x의 형태 == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 결합 벡터를 GRU로 전달
    output, state = self.gru(x)

    # 형태 == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x의 형태 == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # 출력 형태 == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [0]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## 체크포인트

In [0]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [0]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

## 훈련

* 각 `.npy` 파일에 저장된 특성을 추출한 후에 특성들을 인코더를 통해 전달합니다.
* 인코더 출력, 은닉 상태(0으로 초기화된) 및 디코더 입력(시작 토큰)이 디코더로 전달 됩니다.
* 디코더는 예측값과 은닉 상태 디코더를 반환합니다.
* 은닉 상태 디코더는 다시 모델로 전달되고, 그 예측값을 통해 손실을 계산합니다.
* 디코더의 다음 입력을 결정할 때는 Teacher-forcing 을 이용합니다.
* Teacher-forcing 은 타깃 단어가 디코더에 다음 입력으로 전달되는 기술입니다.
* 마지막 단계는 그래디언트를 계산하고 옵티마이저와 역전파에 적용하는 것 입니다.

In [8]:
# 훈련 셀을 여러 번 실행하면 loss_plot 배열이 재설정되므로
# 이를 별도의 분리된 셀에 추가합니다.
loss_plot = []

In [0]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # 캡션은 이미지와는 관계가 없기 때문에
  # 각 묶음의 은닉 상태를 초기화합니다.
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # 디코더를 통해 특성을 전달
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # Teacher-forcing 사용
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [0]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # 나중에 플롯할 에포크의 끝과 손실값을 저장합니다.
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## 캡션!

* 평가 함수는 훈련 루프와 유사하지만, 여기서는 Teacher-forcing을 사용하지 않는다는 점을 제외합니다. 각 시간 단계에 따른 디코더에 대한 입력은 은닉 상태와 인코더 출력과 마찬가지로 이전의 예측입니다. 
* 모델이 최종 토큰을 예측하는 경우 예측을 중지하세요.
* 그 다음 매 단계마다 초점 가중치를 저장하세요.

In [0]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [0]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [0]:
# 검증 세트의 캡션
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)
# 이미지 열기
Image.open(img_name_val[rid])

## 자신의 이미지로 시도해보기
재미로, 아래에서는 방금 훈련시킨 모델로 자신의 이미지를 캡션하는데 사용하는 방법을 제공했습니다. 상대적으로 적은 양의 데이터로 훈련되었으며, 당신의 이미지는 훈련 데이터와 다를 수 있다는 것을 명심하세요.(따라서 이상한 결과에 대비하세요!)

In [0]:
image_url = 'https://tensorflow.org/images/surf.jpg'
image_extension = image_url[-4:]
image_path = tf.keras.utils.get_file('image'+image_extension,
                                     origin=image_url)

result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# 이미지 열기
Image.open(image_path)

# 다음 단계

축하합니다! 드디어 초점 기반 이미지 캡션 모델을 훈련하셨습니다. 다음으로, 이 예제를 읽어보세요. [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb) 이것은 스페인어와 영어 문장을 번역하기 위해 비슷한 아키텍처를 사용합니다. 이 노트북의 코드를 다른 데이터 세트에서 훈련하는 실험을 해 볼 수도 있습니다.